In [1]:
import sympy as sp
def is_Z_matrix(A):
    n = A.shape[0]
    for i in range(n):
        for j in range(n):
            if i != j and A[i, j] > 0:
                return False
    return True
def is_M_matrix(A):
    n = A.shape[0]
    if not is_Z_matrix(A):
        return False
    # Check all eigenvalues are real and non-negative
    eigvals = A.eigenvals()
    for ev in eigvals:
        ev_val = sp.re(ev.evalf())
        if ev_val < 0:
            return False
    return True

A = sp.Matrix([
    [4, -1, 0],
    [-2, 3, -1],
    [0, -1, 2]
])
print("Is M-matrix?", is_M_matrix(A))
eigenvals= A.eigenvals()
for x in eigenvals:
    if not sp.re(x.evalf()) > 0:
        print("Eigenvalue is not real:", x)


Is M-matrix? True


In [2]:
def multivariate_characteristic_polynomial(A):
    """
    Returns the multivariate characteristic polynomial det(diag(x1,...,xn) - A)
    for a given sympy Matrix A.
    """
    n = A.shape[0]
    lambdas = sp.symbols('x1:%d' % (n+1))  # x1, x2, ..., xn
    diag_lambda = sp.diag(*lambdas)
    char_poly_multi = (diag_lambda + A).det(method='berkowitz')
    #numer, denom = sp.fraction(char_poly_multi)
    poly = sp.Poly(char_poly_multi, *lambdas)
    return poly

In [3]:
def homogenized_multivariate_characteristic_polynomial(A):
    """
    Returns the multivariate characteristic polynomial det(diag(x1,...,xn) - A)
    for a given sympy Matrix A.
    """
    n = A.shape[0]
    lambdas = sp.symbols('x0:%d' % (n+1))  # x1, x2, ..., xn
    diag_lambda = sp.diag(*lambdas[1:])
    char_poly_multi = (diag_lambda +lambdas[0]*A).det(method='berkowitz')
    #numer, denom = sp.fraction(char_poly_multi)
    poly = sp.Poly(char_poly_multi, *lambdas)
    return poly

A= sp.Matrix([
    [2, -1],
    [-1, 2]
])
print("Matrix A:")
sp.pprint(A)
poly1= multivariate_characteristic_polynomial(A)
print("Multivariate characteristic polynomial:")
sp.pprint(poly1)
poly = homogenized_multivariate_characteristic_polynomial(A)
print("Homogenized multivariate characteristic polynomial:")
sp.pprint(poly)

Matrix A:
⎡2   -1⎤
⎢      ⎥
⎣-1  2 ⎦
Multivariate characteristic polynomial:
Poly(x1*x2 + 2*x1 + 2*x2 + 3, x1, x2, domain='ZZ')
Homogenized multivariate characteristic polynomial:
Poly(3*x0**2 + 2*x0*x1 + 2*x0*x2 + x1*x2, x0, x1, x2, domain='ZZ')


In [20]:
import sympy as sp
import random
import Garding as gd
import numpy as np
random.seed(42)
def random_nonneg_matrix(n, min_val=0, max_val=100,sparsity=1):
    return sp.Matrix([[random.randint(min_val, max_val) if random.random()<=sparsity else 0 for _ in range(n)] for _ in range(n)])
    #return sp.Matrix([[random.randint(min_val, max_val) for _ in range(n)] for _ in range(n)])
def is_Z_matrix(A):
    n = A.shape[0]
    for i in range(n):
        for j in range(n):
            if i != j and A[i, j] > 0:
                return False
    return True

def is_M_matrix(A):
    n = A.shape[0]
    if not is_Z_matrix(A):
        return False
    # Check all eigenvalues are real and non-negative
    eigvals = A.eigenvals()
    for ev in eigvals:
        ev_val = sp.re(ev.evalf())
        if ev_val < 0:
            return False
    return True

def power_iteration_spectral_radius(A, num_simulations: int = 10000):
    # Convert sympy Matrix to numpy array of floats
    if isinstance(A, sp.Matrix):
        A = np.array(A).astype(np.float64)
    b_k = np.random.rand(A.shape[1])
    for _ in range(num_simulations):
        b_k1 = np.dot(A, b_k)
        b_k1_norm = np.linalg.norm(b_k1)
        if b_k1_norm == 0:
            # All-zero vector, spectral radius is zero
            return 0.0
        b_k = b_k1 / b_k1_norm
    return np.linalg.norm(np.dot(A, b_k))

def power_iteration_spectral_radius1(A, num_simulations: int = 10000):
    # Convert sympy Matrix to numpy array of floats
    if isinstance(A, sp.Matrix):
        A = np.array(A).astype(np.float64)
    A=np.dot(A,A.transpose())
    b_k = A
    for _ in range(num_simulations):
        b_k1 = np.dot(A, b_k)
        b_k1_norm = np.linalg.norm(b_k1)
        b_k = b_k1 / b_k1_norm
    spectral_radius=np.sqrt(np.linalg.norm(np.dot(A, b_k)))
    return spectral_radius

def multivariate_characteristic_polynomial(A):
    """
    Returns the multivariate characteristic polynomial det(diag(x1,...,xn) - A)
    for a given sympy Matrix A.
    """
    n = A.shape[0]
    lambdas = sp.symbols('x1:%d' % (n+1))  # x1, x2, ..., xn
    diag_lambda = sp.diag(*lambdas)
    char_poly_multi = (diag_lambda + A).det(method='berkowitz')
    #numer, denom = sp.fraction(char_poly_multi)
    poly = sp.Poly(char_poly_multi, *lambdas)
    return poly
def generate_random_M_matrix(n, min_val=0, max_val=100,sparsity=1):
    A = random_nonneg_matrix(n, min_val, max_val,sparsity=sparsity)
    spectral_radius = power_iteration_spectral_radius(A)
    s=random.randint(int(spectral_radius),2*int(spectral_radius))+1
    M = (s * sp.eye(n) - A)
    return M

def homog_inverse_multi_var_char_poly(A):
    n = A.shape[0]
    lambdas = sp.symbols('x0:%d' % (n+1))  # x1, x2, ..., xn
    diag_lambda = sp.diag(*lambdas[1:])
    char_poly_multi = (lambdas[0]*sp.eye(n)+ diag_lambda*A).det(method='berkowitz')
    #numer, denom = sp.fraction(char_poly_multi)
    poly = sp.Poly(char_poly_multi, *lambdas)
    return poly


def random_positive_expansion(q, vars):
    n = len(vars)
    # Random positive direction v and point x
    v = [random.randint(0, 40) for _ in range(n)]
    x = [random.randint(0, 1000) for _ in range(n)]
    t = sp.symbols('t')
    subs = {vars[i]: v[i]*t + x[i] for i in range(n)}
    g = q.subs(subs)
    poly= sp.Poly(g, t)
    return v, x, poly

A= sp.Matrix([
    [2, -1],
    [-1, 2]
])
print("Matrix A:")
sp.pprint(A)
poly1= multivariate_characteristic_polynomial(A)
print("Multivariate characteristic polynomial:")
sp.pprint(poly1)
poly = homog_inverse_multi_var_char_poly(A)
print("Homogenized inverse multivariate characteristic polynomial:")
sp.pprint(poly)
v,x,g=random_positive_expansion(poly, poly.gens)
print("is g U-stable?", gd.UpsilonStable(g.coeffs()))

n=8
count=0
for i in range(10):
    M = generate_random_M_matrix(n,min_val=0, max_val=100,sparsity=1)
    poly = homog_inverse_multi_var_char_poly(M)
    var=poly.gens
    poly=sp.diff(poly, var[0]).subs({var[0]:1})
    for _ in range(10):
        v,x,g=random_positive_expansion(poly, poly.gens)
        #print(g.coeffs())
        if not gd.realstable_univarite(g.coeffs()):
            count+=1
        if not gd.UpsilonStable(g.coeffs()):
            print("Matrix M:")
            sp.pprint(M)
            print("Polynomial:")
            sp.pprint(poly)
            print("Expansion g(t):")
            sp.pprint(g)
            print("Direction v:", v)
            print("Point x:", x)
print(f"Number of non-real stable: {count}")
# Example: generate a 4x4 matrixd

#auto_check(n=7,trials=1)

Matrix A:
⎡2   -1⎤
⎢      ⎥
⎣-1  2 ⎦
Multivariate characteristic polynomial:
Poly(x1*x2 + 2*x1 + 2*x2 + 3, x1, x2, domain='ZZ')
Homogenized inverse multivariate characteristic polynomial:
Poly(x0**2 + 2*x0*x1 + 2*x0*x2 + 3*x1*x2, x0, x1, x2, domain='ZZ')
is g U-stable? True
Number of non-real stable: 0


In [ ]:
M = generate_random_M_matrix(n,sparsity=0.2)
poly = homog_inverse_multi_var_char_poly(M)
print("Matrix M:")
sp.pprint(M)
print("Homogenized inverse multivariate characteristic polynomial:")
sp.pprint(poly)
for _ in range(10):
        v,x,g=random_positive_expansion(poly, poly.gens)
        if not gd.UpsilonStable(g.coeffs()):
            print("Matrix M:")
            sp.pprint(M)
            print("Polynomial:")
            sp.pprint(poly)
            print("Expansion g(t):")
            sp.pprint(g)
            print("Direction v:", v)
            print("Point x:", x)

Matrix M:
⎡60    0    0    0   0   -25⎤
⎢                           ⎥
⎢ 0   60   -45   0   0    0 ⎥
⎢                           ⎥
⎢-49  -29  60    0   0    0 ⎥
⎢                           ⎥
⎢ 0    0    0   60   0    0 ⎥
⎢                           ⎥
⎢ 0    0    0   -71  49  -2 ⎥
⎢                           ⎥
⎣ 0    0   -55  -6   0   60 ⎦
Homogenized inverse multivariate characteristic polynomial:
Poly(x0**6 + 60*x0**5*x1 + 60*x0**5*x2 + 60*x0**5*x3 + 60*x0**5*x4 + 49*x0**5* ↪

↪ x5 + 60*x0**5*x6 + 3600*x0**4*x1*x2 + 3600*x0**4*x1*x3 + 3600*x0**4*x1*x4 +  ↪

↪ 2940*x0**4*x1*x5 + 3600*x0**4*x1*x6 + 2295*x0**4*x2*x3 + 3600*x0**4*x2*x4 +  ↪

↪ 2940*x0**4*x2*x5 + 3600*x0**4*x2*x6 + 3600*x0**4*x3*x4 + 2940*x0**4*x3*x5 +  ↪

↪ 3600*x0**4*x3*x6 + 2940*x0**4*x4*x5 + 3600*x0**4*x4*x6 + 2940*x0**4*x5*x6 +  ↪

↪ 137700*x0**3*x1*x2*x3 + 216000*x0**3*x1*x2*x4 + 176400*x0**3*x1*x2*x5 + 2160 ↪

↪ 00*x0**3*x1*x2*x6 + 216000*x0**3*x1*x3*x4 + 176400*x0**3*x1*x3*x5 + 148625*x ↪

↪ 0**3*x1*x3*x6 + 176400*x

In [ ]:
A=generate_random_M_matrix(5,sparsity=1)
print("Generated M-matrix A:")
sp.pprint(A)
print("spectral radius of A:", power_iteration_spectral_radius(A))
print("power_iteration_spectral_radius1 of A:", power_iteration_spectral_radius1(A))
eigenvals= A.eigenvals()
print("spectral radius of A:", max([sp.Abs(x.evalf()) for x in eigenvals]))

Generated M-matrix A:
⎡605  -96  -60  -37  -87⎤
⎢                       ⎥
⎢-97  613  -7   -35  -95⎥
⎢                       ⎥
⎢-79  -29  617  -68  -15⎥
⎢                       ⎥
⎢-80  -87  -43  623  -23⎥
⎢                       ⎥
⎣-19  -9   -92  -87  651⎦
spectral radius of A: 724.7017358438686
power_iteration_spectral_radius1 of A: 726.0778407863204
spectral radius of A: 724.701735843869


In [ ]:
def auto_check(n=6,trials=50):
    flag=True
    for _ in range(trials):
        M = generate_random_M_matrix(n)
        poly = multivariate_characteristic_polynomial(M)
        y=sp.symbols('y')
        var=(y,)+poly.gens
        homogenized_poly = gd.homogenize_poly(poly, n, n,var)
        #homogenized_poly =homogenized_multivariate_characteristic_polynomial(M)
        if not gd.is_Lorentzian(homogenized_poly,homogenized_poly.gens, M_convex=True):
            print("Counterexample found!")
            print("Random non-negative matrix:")
            sp.pprint(A)
            print("Spectral radius of A:", spectral_radius)
            print("Shifted matrix M = sI - A:", M)
            print("Is M-matrix?", is_M_matrix(M))
            print("Multivariate characteristic polynomial:")
            sp.pprint(poly)
            print("Homogenized polynomial:",homogenized_poly)
            flag=False
            return flag
    if flag:
        print("All tests passed!")
    return flag

In [ ]:
def degree_leq_2_multivariate_charpoly(A):
    """
    Efficiently compute the degree <= 2 part of the multivariate characteristic polynomial of A.
    """
    n = A.shape[0]
    if isinstance(A, sp.Matrix):
        A = np.array(A).astype(np.float64)
    xs = sp.symbols('x1:%d' % (n+1))
    result = A
    const_term = np.linalg.det(result)

    poly = const_term

    # Linear terms
    for i in range(n):
        minor = A.copy()
        minor = np.delete(minor, i, axis=0)
        minor = np.delete(minor, i, axis=1)
        coeff =  np.linalg.det(minor) if n > 1 else 1
        poly += xs[i] * coeff

    # Quadratic terms
    for i in range(n):
        for j in range(i+1, n):
            minor = A.copy()
            minor=np.delete(minor, j, axis=0)
            minor=np.delete(minor, i, axis=0)
            minor=np.delete(minor, j, axis=1)
            minor=np.delete(minor, i, axis=1)
            coeff =  np.linalg.det(minor) if n > 2 else 1
            poly += xs[i] * xs[j] * coeff

    return sp.Poly(poly, *xs)

def auto_degree_leq_2_part_of_multivariate_char_polynomial(A):
    """
    Returns the degree <= 2 part of the multivariate characteristic polynomial of A.
    """
    poly = multivariate_characteristic_polynomial(A)
    # Get the dictionary representation: {monomial_exponents: coefficient}
    poly_dict = poly.as_dict()
    # Filter for degree <= 2
    filtered_dict = {exp: coeff for exp, coeff in poly_dict.items() if sum(exp) <= 2}
    # Reconstruct the polynomial
    return sp.Poly.from_dict(filtered_dict, poly.gens)
n=6
A = generate_random_M_matrix(n)
deg2_poly= degree_leq_2_multivariate_charpoly(A)
auto_deg2_poly = auto_degree_leq_2_part_of_multivariate_char_polynomial(A)
print(auto_deg2_poly)
print(deg2_poly)
if deg2_poly == auto_deg2_poly:
    print("The two methods produce the same degree <= 2 polynomial.")

Poly(24706493906*x1*x2 + 22973782644*x1*x3 + 22907944910*x1*x4 + 25054640514*x1*x5 + 26007853732*x1*x6 + 9209940540284*x1 + 25641454460*x2*x3 + 25410881829*x2*x4 + 28554482052*x2*x5 + 29308252456*x2*x6 + 10294187433988*x2 + 23216898495*x3*x4 + 26563035622*x3*x5 + 27480790760*x3*x6 + 9238068111966*x3 + 26906708014*x4*x5 + 27055722591*x4*x6 + 9272561700825*x4 + 31468356852*x5*x6 + 10755106092012*x5 + 11166168247654*x6 + 3654829981925098, x1, x2, x3, x4, x5, x6, domain='ZZ')
Poly(24706493906.0*x1*x2 + 22973782644.0*x1*x3 + 22907944910.0*x1*x4 + 25054640514.0*x1*x5 + 26007853732.0*x1*x6 + 9209940540283.97*x1 + 25641454460.0*x2*x3 + 25410881829.0*x2*x4 + 28554482052.0*x2*x5 + 29308252456.0*x2*x6 + 10294187433988.0*x2 + 23216898495.0*x3*x4 + 26563035622.0*x3*x5 + 27480790760.0*x3*x6 + 9238068111965.97*x3 + 26906708014.0*x4*x5 + 27055722591.0*x4*x6 + 9272561700825.0*x4 + 31468356852.0*x5*x6 + 10755106092012.0*x5 + 11166168247654.0*x6 + 3.6548299819251e+15, x1, x2, x3, x4, x5, x6, domain='RR')

下面的程序自动验证一个随机生成的 M-matrix 的齐次化多重特征多项式
$$f(x_0,x_1,\cdots,x_n):=\det(\text{diag}\{x_1,\cdots,x_n\}+x_0 M)$$
是否是一个 Lorentzian 多项式。

目前为止所有的结果都是肯定的。
计算时间
|$n$|计算一次的平均时间|
|---|------|
|6|0.08s|
|10|0.15s|
|20|0.80s|
|40|8.40s|
|50|18.90s|
|60|36.20s|


In [ ]:
def auto_check(n=6,trials=1,sparsity=1,M_matrix=True):
    flag=True
    for _ in range(trials):
        if M_matrix:
            M = generate_random_M_matrix(n,sparsity=sparsity)
        else:
            M = random_nonneg_matrix(n, min_val=0, max_val=10,sparsity=sparsity)
        poly = degree_leq_2_multivariate_charpoly(M)
        y=sp.symbols('y')
        var=(y,)+poly.gens
        homogenized_poly = gd.homogenize_poly(poly, n, n,var)
        test_poly =homogenized_poly
        for _ in range(n-2):
            test_poly = sp.diff(test_poly, y)
        #homogenized_poly =homogenized_multivariate_characteristic_polynomial(M)
        if not gd.is_Lorentzian(test_poly,test_poly.gens, M_convex=True):
            print("Counterexample found!")
            print("M-matrix:")
            sp.pprint(M)
            print("Is M-matrix?", is_M_matrix(M))
            print("Multivariate characteristic polynomial:")
            sp.pprint(poly)
            print("Homogenized polynomial:",homogenized_poly)
            flag=False
        
    if flag:
        print("All tests passed!")
    return flag
count=0
for _ in range(1):
    if auto_check(n=50,trials=2,sparsity=1,M_matrix=True) ==False:
        count+=1
print("Number of failures:", count)
    


All tests passed!
Number of failures: 0


In [ ]:
import numpy as np
def power_iteration(A, num_simulations: int = 100):
    b_k = np.random.rand(A.shape[1])
    for _ in range(num_simulations):
        b_k1 = np.dot(A, b_k)
        b_k1_norm = np.linalg.norm(b_k1)
        b_k = b_k1 / b_k1_norm
    return np.linalg.norm(np.dot(A, b_k))
# Usage:
spectral_radius = power_iteration(A)

TypeError: loop of ufunc does not support argument 0 of type Float which has no callable sqrt method

In [ ]:
n=6
A = random_nonneg_matrix(n)
eigvals = A.eigenvals()
spectral_radius = max(abs(ev.evalf()) for ev in eigvals)
s=random.randint(int(spectral_radius)+1,4*int(spectral_radius)+1)
M = (s * sp.eye(n) - A)
#poly = homogenized_multivariate_characteristic_polynomial(M)

print("Random non-negative matrix A:")
sp.pprint(A)
print("Spectral radius of A:", spectral_radius)
print("Shifted matrix M = sI - A:", M)
#print("Is M-matrix?", is_M_matrix(M))
#print("Homogeneous Multivariate characteristic polynomial:")
#sp.pprint(poly)

#homogenized_poly = gd.homogenize_poly(poly, n, n,var)

Random non-negative matrix A:
⎡6   21  14  2   30  5 ⎤
⎢                      ⎥
⎢15  22  9   19  4   0 ⎥
⎢                      ⎥
⎢0   19  28  24  4   25⎥
⎢                      ⎥
⎢23  9   17  1   6   15⎥
⎢                      ⎥
⎢25  28  27  11  2   28⎥
⎢                      ⎥
⎣6   12  11  1   9   29⎦
Spectral radius of A: 82.0772890996765
Shifted matrix M = sI - A: Matrix([[323, -21, -14, -2, -30, -5], [-15, 307, -9, -19, -4, 0], [0, -19, 301, -24, -4, -25], [-23, -9, -17, 328, -6, -15], [-25, -28, -27, -11, 327, -28], [-6, -12, -11, -1, -9, 300]])


In [ ]:
t, s = sp.symbols('t s')
poly = t**2 + t + 1
print("Polynomial:", poly)
vars = sp.symbols('t s')
hpoly = gd.homogenize_poly(poly, 2, 2, vars)

Polynomial: t**2 + t + 1


ValueError: too many values to unpack (expected 1)

In [ ]:
import sympy as sp
import Garding as gd
from Garding import poly_pretreat
def homogenize_poly(poly, n, d, variables=None):
    """
    Homogenize a polynomial to degree d in n variables by adding a new variable x0.
    """
    if variables is None:
        variables = sp.symbols('x0:%d' % (n+1))
    x0 = variables[0]
    new_vars = variables[1:]
    new_poly_dict = {}
    if isinstance(poly, sp.Expr):
        poly = sp.Poly(poly, *new_vars)
        poly_dict = poly.as_dict()
    elif isinstance(poly, sp.Poly):
        poly_dict = poly.as_dict()
        # Already in Poly form
    elif isinstance(poly, dict):
        poly_dict = poly
    
    for exps, coeff in poly_dict.items():
        total_deg = sum(exps)
        if total_deg > d:
            raise ValueError("Polynomial degree exceeds target degree for homogenization.")
        new_exps = (d - total_deg,) + exps  # Add exponent for x0
        new_poly_dict[new_exps] = coeff
    return sp.Poly.from_dict(new_poly_dict, (x0,) + new_vars)
y,t , s = sp.symbols('y, t s')
poly = t**2*s + t*s**5 + 4
print("Polynomial:", poly)
vars = sp.symbols('s t')

hpoly = homogenize_poly(poly, 2, 7, (y,t,s))
print("Homogenized polynomial:", hpoly)

Polynomial: s**5*t + s*t**2 + 4
Homogenized polynomial: Poly(4*y**7 + y**4*t**2*s + y*t*s**5, y, t, s, domain='ZZ')


In [ ]:
import sympy as sp

# Example usage:
A = sp.Matrix([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])
poly = multivariate_characteristic_polynomial(A)
print("Multivariate characteristic polynomial:")
sp.pprint(poly)

Multivariate characteristic polynomial:
-x₁⋅x₂⋅x₃ + 9⋅x₁⋅x₂ + 5⋅x₁⋅x₃ + 3⋅x₁ + x₂⋅x₃ + 12⋅x₂ + 3⋅x₃


In [4]:
import numpy as np
import sympy as sp
import Garding as gd
x,y= sp.symbols('x y')
a=2/3
f=((x+a)**2+(y+a)**2+sp.sqrt(3)*(x+a)*(y+a)-1)*(x+a)*(y+a)
f=sp.Poly(f,x,y)
print("Polynomial f(x,y):",f.as_expr())
# Use the notebook's local homogenize_poly (defined in another cell) instead of gd.homogenize_poly
# to avoid the UnboundLocalError originating inside gd.homogenize_poly.
hpoly = gd.homogenize_poly(f, 2, 4, (sp.symbols('x0'), x, y))
print("Homogenized polynomial:", hpoly.as_expr(), hpoly.gens)
print("Is Lorentzian?", gd.is_Lorentzian(hpoly, hpoly.gens))

Polynomial f(x,y): x**3*y + 0.666666666666667*x**3 + sqrt(3)*x**2*y**2 + x**2*y*(2.0 + 1.33333333333333*sqrt(3)) + x**2*(0.444444444444444*sqrt(3) + 1.33333333333333) + x*y**3 + x*y**2*(2.0 + 1.33333333333333*sqrt(3)) + x*y*(1.66666666666667 + 1.77777777777778*sqrt(3)) + x*(0.518518518518518 + 0.592592592592593*sqrt(3)) + 0.666666666666667*y**3 + y**2*(0.444444444444444*sqrt(3) + 1.33333333333333) + y*(0.518518518518518 + 0.592592592592593*sqrt(3)) - 0.0493827160493827 + 0.197530864197531*sqrt(3)
Homogenized polynomial: 0.666666666666667*x**3*x0 + x**3*y + x**2*x0**2*(0.444444444444444*sqrt(3) + 1.33333333333333) + x**2*x0*y*(2.0 + 1.33333333333333*sqrt(3)) + sqrt(3)*x**2*y**2 + x*x0**3*(0.518518518518518 + 0.592592592592593*sqrt(3)) + x*x0**2*y*(1.66666666666667 + 1.77777777777778*sqrt(3)) + x*x0*y**2*(2.0 + 1.33333333333333*sqrt(3)) + x*y**3 + x0**4*(-0.0493827160493827 + 0.197530864197531*sqrt(3)) + x0**3*y*(0.518518518518518 + 0.592592592592593*sqrt(3)) + x0**2*y**2*(0.444444444444